[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/jolin-io/fall-in-love-with-julia/main?filepath=10%20SciML%20-%2001%20DiffEqFlux.ipynb)

<a href="https://www.jolin.io" target="_blank" rel="noreferrer noopener">
<img src="https://www.jolin.io/assets/Jolin/Jolin-Banner-Website-v1.1-darkmode.webp">
</a>

# Fall-in-love-with-Julia: SciML and DiffEqFlux.jl

a 101 introduction session


I am Stephan Sahm, and here is what we are going to learn today

1. Flux.jl - Neural Networks
2. DifferentialEquations.jl
3. SciML - DiffEq within NeuralNet
4. SciML - NeuralNet within DiffEq
5. Turing.jl - Uncertainity Modelling via Bayesian Estimation
6. SciML - Bayesian DiffEq
7. SciML - DataDrivenDiffEq.jl and Symbolic Regression


The main sources of information for this jupyter notebook are the following two tutorials
- [Blog Post DiffEqFlux.jl](https://julialang.org/blog/2019/01/fluxdiffeq/)
- [Tutorial Bayesian Differential Equations](https://turing.ml/dev/tutorials/10-bayesian-differential-equations/)

In [ ]:
using Plots
using StatsPlots
using Statistics
using Random
using LinearAlgebra

# Flux.jl - Neural Networks

In [ ]:
import Flux

In [ ]:
function goal(x)
    s = sum(x)
    s > length(x)/2
end

In [ ]:
train_x = [rand(10) for i in 1:1000];
train_y = goal.(train_x);

val_x = [rand(10) for i in 1:100];
val_y = goal.(val_x);

p1 = histogram(sum.(train_x)[train_y.==0], label=:0, color=:red)
p2 = histogram(sum.(train_x)[train_y.==1], label=:1, color=:green)
plot(p1, p2, layout=(2,1), link=:x, xlabel=:sum)

In [ ]:
neural_net = Flux.Chain(
    x -> x.^3,
    Flux.Dense(10 => 5, Flux.σ),
    Flux.Dense(5 => 2),
    Flux.softmax
)
neural_net(rand(10)) # => 2-element vector

In [ ]:
_accuracy(x, y) = Flux.onecold(neural_net(x), 0:1) == y
accuracy(x, y) = mean(_accuracy.(x, y))

In [ ]:
accuracy(train_x, train_y), accuracy(val_x, val_y)

In [ ]:
loss(x, y) = Flux.crossentropy(neural_net(x), Flux.onehot(y, 0:1))
opt = Flux.Momentum(0.01)
params = Flux.params(neural_net)

for (x, y) in zip(train_x, train_y)
    gs = Flux.gradient(params) do
        loss(x, y)
    end
    Flux.update!(opt, params, gs)
end
accuracy(train_x, train_y), accuracy(val_x, val_y)

**🫵 It is your time:** create a plot similar to the histogram above in order to visualize how well our neural network worked

In [ ]:
# your space

# DifferentialEquations.jl

Example [Lotka-Volterra equations](https://en.wikipedia.org/wiki/Lotka%E2%80%93Volterra_equations): Population of rabbits and foxes

<center>

rabbits: $ x^\prime = \alpha x - \beta x y $

</center>

the rate of change of the prey's population is given by its own growth rate ($\alpha$) minus the rate at which it is preyed upon ($\beta$).


<center>

foxes: $ y^\prime = \gamma x y - \delta y $

</center>

the rate of change of the predator's population depends upon the rate at which it consumes prey ($\gamma$), minus its intrinsic death rate ($\delta$)


In [ ]:
using DifferentialEquations
function lotka_volterra(du, u, p, t)
    x, y = u
    α, β, δ, γ = p
    du[1] = dx = α*x - β*x*y
    du[2] = dy = -δ*y + γ*x*y
end
u0 = [1.0, 1.0]
tspan = (0.0, 10.0)
p = [1.5, 1.0, 3.0, 1.0]
ode_prob = ODEProblem(lotka_volterra, u0, tspan, p)

In [ ]:
ode_sol = solve(ode_prob)
plot(ode_sol)

# SciML - DiffEq within NeuralNet

In [ ]:
import Flux, DiffEqFlux
using DifferentialEquations

p = [2.2, 1.0, 2.0, 0.4] # Initial Parameter Vector
params = Flux.params(p)
function predict_rd() # Our 1-layer "neural network"
    # override with new parameters
    solve(ode_prob, Tsit5(), p=p, saveat=0.1)[1,:]
end
loss_rd() = sum(abs2, x-1 for x in predict_rd())


data = Iterators.repeated((), 100)
opt = Flux.ADAM(0.1)
cb = function () #callback function to observe training
    # display(loss_rd())
    # using `remake` to re-create our `ode_prob` with current parameters `p`
    sleep(0.05)
    plot(solve(remake(ode_prob,p=p), Tsit5(), saveat=0.1), ylim=(0,6), show=:inline)
end

# Display the ODE with the initial parameter values.
cb();
Flux.train!(loss_rd, params, data, opt, cb=cb)

an ODE can be part of a larger neural network
```julia 
neuralnet = Chain(
    Dense(28^2, 32, relu),
    # requires an ODE of 32 params
    p -> solve(ode_prob, Tsit5(), p=p,
               saveat=0.1)[1,:],
    Dense(32, 10),
    softmax)
```

**🫵 It is your time:** create data from an alternative Lotka Volterra system and fit it 

In [ ]:
# your space

# SciML - NeuralNet within DiffEq

Ground Truth $u^\prime = A u^3$

In [ ]:
using DifferentialEquations

function trueODEfunc(du,u,p,t)
    true_A = [-0.1 -2.0
               2.0 -0.1]
    du .= true_A * u.^3
end

u0 = Float32[2.; 0.]
datasize = 30
tspan = (0.0f0, 1.5f0)

t = range(tspan[1],tspan[2],length=datasize)
ode2_prob = ODEProblem(trueODEfunc,u0,tspan)
ode2_sol = solve(ode2_prob,Tsit5(),saveat=t)
ode2_data = Array(ode2_sol)

Model $u^\prime$ with neural network.
(multilayer perceptron with 1 hidden layer and a tanh activation function)

In [ ]:
import Flux, DiffEqFlux
dudt = Flux.Chain(
    x -> x.^3,
    Flux.Dense(2, 50, tanh),
    Flux.Dense(50, 2),
)

n_ode = DiffEqFlux.NeuralODE(dudt, tspan, Tsit5(), saveat=t,
                  reltol=1e-7, abstol=1e-9)
ps = Flux.params(n_ode)

# Get the prediction using the correct initial condition
pred = n_ode(u0);

In [ ]:
scatter(t, ode2_data[1,:], label="data")
scatter!(t, pred[1,:], label="prediction")

In [ ]:
loss_n_ode() = sum(abs2, ode2_data .- n_ode(u0))
loss_n_ode()

In [ ]:
data = Iterators.repeated((), 100)
opt = Flux.ADAM(0.1)
cb = function () #callback function to observe training
    # plot current prediction against data
    cur_pred = n_ode(u0)
    pl = scatter(t, ode2_data[1,:], label="data")
    scatter!(pl, t, cur_pred[1,:], label="prediction")
    plot(pl, show=:inline)
end

# Display the ODE with the initial parameter values.
cb()
Flux.train!(loss_n_ode, ps, data, opt, cb = cb)
loss_n_ode()

**🫵 It is your time:** try fitting Lotka Volterra data

In [ ]:
# your space

### Alternative perspective: Paper Neural Ordinary Differential Equations (Chen et al. 2019)

Residual Neural Network (discrete difference layers)
$$h_{t+1} = h_t + f(h_t, \theta_t)$$

Neural Ordinary Differential Equations
$$\frac{dh(t)}{dt} = f(h(t), t, \theta)$$

![](https://www.jolin.io/assets/examples/NeuralODE-Comparing-ResNet.png)


# Turing.jl - Uncertainty Modelling via Bayesian Estimation

Coin Flip mini example

find more details at https://turing.ml/dev/tutorials/00-introduction/

In [ ]:
using Turing

# Set the true probability of heads in a coin.
p_true = 0.5

# Iterate from having seen 0 observations to 100 observations.
N = 100

# Draw data from a Bernoulli distribution, i.e. draw heads or tails.
Random.seed!(12)
data = rand(Bernoulli(p_true), N)

# Declare our Turing model.
@model function coinflip(y)
    # Our prior belief about the probability of heads in a coin.
    p ~ Beta(1, 1)

    # The number of observations.
    yN = length(y)
    for n in 1:yN
        # Heads or tails of a coin are drawn from a Bernoulli distribution.
        y[n] ~ Bernoulli(p)
    end
end

# Settings of the Hamiltonian Monte Carlo (HMC) sampler.
iterations = 1000
ϵ = 0.05
τ = 10

# Start sampling.
chain = sample(coinflip(data), HMC(ϵ, τ), iterations)
plot(chain)

In [ ]:
histogram(chain[:p])

**🫵 It is your time:** Try different `N` above and see how our information about `p` improves/worsens

In [ ]:
# your space

# SciML - Bayesian DiffEq

Let's assume noisy Lotka Volterra data

In [ ]:
ode_sol = solve(ode_prob, Tsit5(), saveat=0.1)
ode_data = (Array(ode_sol) + 0.8
           * randn(size(Array(ode_sol))))
# Plot simulation & noisy observations
plot(ode_sol, alpha=0.3)
scatter!(ode_sol.t, ode_data',
         color=[1 2], label="")

Let's assume we only have predator-data (foxes)

In [ ]:
using Turing
using DifferentialEquations

In [ ]:
@model function fitlv(data::AbstractVector, ode_prob)
    # Prior distributions.
    α ~ truncated(Normal(1.5, 0.5), 0.5, 2.5)
    β ~ truncated(Normal(1.2, 0.5), 0, 2)
    γ ~ truncated(Normal(3.0, 0.5), 1, 4)
    δ ~ truncated(Normal(1.0, 0.5), 0, 2)
    p = [α, β, γ, δ]
    
    # Simulate Lotka-Volterra model but save only
    # the second state of the system (predators).
    predicted = solve(ode_prob, Tsit5(), p=p, saveat=0.1, save_idxs=2)
    
    # Observations of the predators.
    σ ~ InverseGamma(2, 3)
    data ~ MvNormal(predicted.u, σ^2 * I)
    return nothing
end

# fit model only to predators (foxes)
model = fitlv(ode_data[2, :], ode_prob)

Sample & plot (called data retroduction)

In [ ]:
# Sample 3 independent chains.
chain = sample(model, NUTS(0.45), MCMCSerial(), 5000, 3, progress=false)
posterior_samples = sample(chain[[:α, :β, :γ, :δ]], 300, replace=false)

plot(legend=false)
for p in eachrow(Array(posterior_samples))
    ode_sol_p = solve(ode_prob, Tsit5(), p=p, saveat=0.1)
    plot!(ode_sol_p, alpha=0.1, color="#BBBBBB")
end

# Plot simulation and noisy observations.
plot!(ode_sol, color=[1 2], linewidth=1)
scatter!(ode_sol.t, ode_data', color=[1 2])

**🫵 It is your time:** How can we check whether the MCMC Bayesian Estimation converged successfully?

In [ ]:
# your space

# SciML - DataDrivenDiffEq.jl and Symbolic Regression

Extract human readable formula from data via symbolic regression.

In [ ]:
using DataDrivenDiffEq
using ModelingToolkit

f(u) = u.^2 .+ 2.0u .- 1.0
X = randn(1, 100);
Y = f.(X)

In [ ]:
@named problem = DirectDataDrivenProblem(X, Y)

In [ ]:
@variables u

In [ ]:
basis_eqs = monomial_basis([u], 4)

In [ ]:
states = [u]
basis = Basis(basis_eqs, states)

In [ ]:
res = solve(problem, basis, STLSQ())

In [ ]:
print(res)

In [ ]:
print(result(res))

# Further Material

- [Tutorial Deep Learning with Flux - A 30-minutes Blitz](https://fluxml.ai/tutorials/2020/09/15/deep-learning-flux.html)
- [Blog Post DiffEqFlux.jl](https://julialang.org/blog/2019/01/fluxdiffeq/)
- [Documentation DiffEqFlux.jl](https://diffeqflux.sciml.ai/stable/)
- [Paper Neural Ordinary Differential Equations (Chen et al. 2019)](https://arxiv.org/abs/1806.07366)
- [Paper Universal Differential Equations for SciML (Rackauckas et al. 2020)](https://arxiv.org/abs/2001.04385)
- [Tutorial Bayesian Differential Equations](https://turing.ml/dev/tutorials/10-bayesian-differential-equations/)
- [Documentation DataDrivenDiffEq.jl](https://datadriven.sciml.ai/stable), [linear ODE example](https://datadriven.sciml.ai/stable/examples/2_linear_continuous_system/), [nonlinear ODE example](https://datadriven.sciml.ai/stable/examples/4_nonlinear_continuous_system/)


# Thank you for joining

for questions or suggestions please contact me at stephan.sahm@jolin.io


#### Sponsored by [Jolin.io](https://www.jolin.io)

Jolin.io is an IT-consultancy focussing on Julia

We are there to help you, if you want to
- try out Julia at your company, or
- transition Matlab, Fortran, R, Python, etc. to Julia
- or speed up your existing Julia code

<a href="https://www.jolin.io" target="_blank" rel="noreferrer noopener">
<img src="https://www.jolin.io/assets/Jolin/Jolin-Banner-Website-v1.1-darkmode.webp">
</a>